# Main script

The main script consists of two parts:
* Table creator script: it creates the tables needed. To parse the raw data it uses a custom regex serde.
* Query: it performs the query to obtain the number of registers in 2008-10.

### Table creator 

In [1]:
%%writefile table-creater.hql

USE joyvan;

DROP TABLE IF EXISTS posts_sample_external;
CREATE EXTERNAL TABLE posts_sample_external (
Id int,
PostTypeId TINYINT,
CreationDate string,
Tags string,
OwnerUserId int,
ParentId int,
Score int,
FavoriteCount int
)
ROW FORMAT
SERDE 'org.apache.hadoop.hive.serde2.RegexSerDe'
WITH SERDEPROPERTIES (
"input.regex" = '^<row(?=.*\\bId="(\\d+)")(?=.*\\bPostTypeId="(\\d)")(?=.*\\bCreationDate="(\\d{4}-[01]\\d-[0-3]\\dT[0-2]\\d:[0-5]\\d:[0-5]\\d\\.\\d+)")(?=.*\\bTags="([^"]+)")?(?=.*\\bOwnerUserId="(\\d+)")?(?=.*\\bParentId="(\\d+)")?(?=.*\\bScore="(-?\\d+)")(?=.*\\bFavoriteCount="(\\d+)")?.*$'
)
LOCATION '/data/stackexchange1000/posts';

DROP TABLE IF EXISTS posts_sample;
CREATE TABLE posts_sample (
    Id int,
    PostTypeId int,
    CreationDate string,
    Tags string,
    OwnerUserId int,
    ParentId int,
    Score int,
    FavoriteCount int
)
PARTITIONED BY (year int, month string);

SET hive.exec.dynamic.partition.mode=nonstrict;
SET hive.exec.max.dynamic.partitions=2048;
SET hive.exec.max.dynamic.partitions.pernode=256;
SET hive.exec.max.created.files=10000;
SET hive.error.on.empty.partition=true;

INSERT OVERWRITE TABLE posts_sample PARTITION (year, month)
SELECT Id,
PostTypeId,
CreationDate,
Tags,
OwnerUserId,
ParentId,
Score,
FavoriteCount,
substr(CreationDate, 1, 4),
substr(CreationDate, 1, 7)
FROM posts_sample_external
WHERE Id is not NULL;

Overwriting table-creater.hql


In [2]:
! hive -S -f table-creater.hql

## Query

In [3]:
%%writefile query.hql

USE joyvan;
WITH all_partitions as (
SELECT ROW_NUMBER() OVER () as row_num, year, month,  count(*) as num_registers from posts_sample group by year, month
)
SELECT year, month, num_registers from all_partitions where row_num = 99;

Overwriting query.hql


In [4]:
! hive -f query.hql


Logging initialized using configuration in jar:file:/usr/local/apache-hive-1.1.0-bin/lib/hive-common-1.1.0.jar!/hive-log4j.properties
OK
Time taken: 1.21 seconds
Query ID = jovyan_20180701183737_1a944c78-fc2f-4f8c-a244-fc11b5a982ae
Total jobs = 2
Launching Job 1 out of 2
Number of reduce tasks not specified. Estimated from input data size: 1
In order to change the average load for a reducer (in bytes):
  set hive.exec.reducers.bytes.per.reducer=<number>
In order to limit the maximum number of reducers:
  set hive.exec.reducers.max=<number>
In order to set a constant number of reducers:
  set mapreduce.job.reduces=<number>
Starting Job = job_1530461112798_0025, Tracking URL = http://7f802f72479b:8088/proxy/application_1530461112798_0025/
Kill Command = /opt/hadoop/bin/hadoop job  -kill job_1530461112798_0025
Hadoop job information for Stage-1: number of mappers: 1; number of reducers: 1
2018-07-01 18:37:40,075 Stage-1 map = 0%,  reduce = 0%
2018-07-01 18:37:51,554 Stage-1 map = 100%,  